# Alaska Plotting Notebook
This notebook takes data generated from other scripts to generate plots. Plots include the plots that are given in Fair et al., (in prep).

Requires the latest version of Seaborn, otherwise a few of the cells will return errors. If you are unsure if you have the latest version, run the below cell.

In [ ]:
%pip install --upgrade seaborn

In [ ]:
from cartopy import crs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point
import seaborn as sns
import scipy as sp
from scipy.stats import median_abs_deviation as mad
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.ticker import StrMethodFormatter

## Snow Depth Comparisons

Generates Figures 5 and 6 from Fair et al., (in prep), which include along-track plots of snow depth (top row), scatter plot correlations (middle row) and depth residual histograms (bottom row). Separate cells are given for Creamer's Field and Toolik.

### Creamer's Field Snow Depths

In [ ]:
# Read the co-located ICESat-2/UAF data
dfsr = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_442010.csv')
df06 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06_snowdepth_rgt1356_cffl_2022321.csv')
df08 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl08_snowdepth_rgt1356_cffl_2022321.csv')

# Remove filler values from depth data
dfsr['lidar_snow_depth'][dfsr['lidar_snow_depth']>2] = np.nan
df06['lidar_snow_depth'][df06['lidar_snow_depth']>2] = np.nan
dfsr['lidar_snow_depth'][dfsr['lidar_snow_depth']>2] = np.nan
df06['lidar_snow_depth'][df06['lidar_snow_depth']>2] = np.nan

In [ ]:
# Derive r2 values between snow depths
df06_r2 = df06[df06['beam']==5].dropna()
linreg06 = sp.stats.linregress(df06_r2['lidar_snow_depth'], df06_r2['is2_snow_depth'])
df08_r2 = df08[df08['beam']==5].dropna()
linreg08 = sp.stats.linregress(df08_r2['lidar_snow_depth'], df08_r2['is2_snow_depth'])
dfsr_r2 = dfsr[(dfsr['beam']==50)&(dfsr['is2_snow_depth']>0)].dropna()
linregsr = sp.stats.linregress(dfsr_r2['lidar_snow_depth'], dfsr_r2['is2_snow_depth'])

In [ ]:
# Derive weights for residual histograms
w06 = np.ones_like(df06['snow_depth_residual'][df06['beam']==5])/float(len(df06['snow_depth_residual'][df06['beam']==5]))
w08 = np.ones_like(df08['snow_depth_residual'][df08['beam']==5])/float(len(df08['snow_depth_residual'][df08['beam']==5]))
wsr = np.ones_like(dfsr['snow_depth_residual'][(dfsr['beam']==50)])/float(len(dfsr['snow_depth_residual'][(dfsr['beam']==50)]))

# Calculate NMAD of depth residuals
nmad06 = 1.4826*mad(df06['snow_depth_residual'][df06['beam']==5].dropna())
nmad08 = 1.4826*mad(df08['snow_depth_residual'][df08['beam']==5].dropna())
nmadsr = 1.4826*mad(dfsr['snow_depth_residual'][(dfsr['beam']==50)].dropna())

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(12, 9))
## Line plots
#----------------#
# ATL06
ax[0,0].plot(df06['lat'][df06['beam']==5], df06['is2_snow_depth'][df06['beam']==5], label='ICESat-2', linewidth=3)
ax[0,0].plot(df06['lat'][df06['beam']==5], df06['lidar_snow_depth'][df06['beam']==5], label='UAF')
ax[0,0].set_title('ATL06')
ax[0,0].set_ylabel('Snow depth [m]', fontsize=14)
ax[0,0].set_xlim([64.861, 64.884])
ax[0,0].set_ylim([0, 1.75])
ax[0,0].legend(loc='upper right')
ax[0,0].yaxis.set_major_formatter('{x:9<3.1f}')
ax[0,0].set_xticks([64.865, 64.870, 64.875, 64.88])
ax[0,0].tick_params(axis='both', which='major', labelsize=12)
ax[0,0].annotate("(a)", (64.862, 1.5), fontsize=16, bbox=dict(facecolor='white', alpha=0.75))
ax[0,0].grid(True)
# ATL08
ax[0,1].plot(df08['lat'][df08['beam']==5], df08['is2_snow_depth'][df08['beam']==5], label='ICESat-2', linewidth=3)
ax[0,1].plot(df08['lat'][df08['beam']==5], df08['lidar_snow_depth'][df08['beam']==5], label='UAF')
ax[0,1].set_title('ATL08')
ax[0,1].set_xlabel('Latitude', fontsize=14)
ax[0,1].set_xlim([64.861, 64.884])
ax[0,1].set_ylim([0, 1.75])
ax[0,1].yaxis.set_major_formatter('{x:9<3.1f}')
ax[0,1].set_xticks([64.865, 64.870, 64.875, 64.88])
ax[0,1].tick_params(axis='both', which='major', labelsize=12)
ax[0,1].annotate("(b)", (64.862, 1.5), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[0,1].grid(True)
# ATL06-SR
ax[0,2].plot(dfsr['lat'][(dfsr['beam']==50)],
           dfsr['is2_snow_depth'][(dfsr['beam']==50)],
           label='ICESat-2', linewidth=3)
ax[0,2].plot(dfsr['lat'][(dfsr['beam']==50)],
           dfsr['lidar_snow_depth'][(dfsr['beam']==50)],
           label='UAF')
ax[0,2].set_title('ATL06-SR')
ax[0,2].set_xlim([64.861, 64.884])
ax[0,2].set_ylim([0, 1.75])
ax[0,2].yaxis.set_major_formatter('{x:9<3.1f}')
ax[0,2].set_xticks([64.865, 64.870, 64.875, 64.88])
ax[0,2].tick_params(axis='both', which='major', labelsize=12)
ax[0,2].annotate("(c)", (64.862, 1.5), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[0,2].grid(True)

## Scatter plots
#----------------#
# ATL06
sns.scatterplot(data=df06[df06['beam']==5], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[1,0])
ax[1,0].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[1,0].set_xlabel(' ')
ax[1,0].set_ylabel('ICESat-2 snow depth [m]', fontsize=14)
ax[1,0].set_xlim([0, 1.5])
ax[1,0].set_ylim([0, 1.5])
ax[1,0].xaxis.set_major_formatter('{x:9<3.1f}')
ax[1,0].yaxis.set_major_formatter('{x:9<3.1f}')
ax[1,0].tick_params(axis='both', which='major', labelsize=12)
ax[1,0].annotate("r$^2$ = {:.2f}".format(linreg06.rvalue), (0.8, 0.15), fontsize=12)
ax[1,0].annotate("(d)", (0.05, 1.3), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[1,0].grid(True, alpha=0.3)
# ATL08
sns.scatterplot(data=df08[df08['beam']==5], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[1,1])
ax[1,1].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[1,1].set_xlabel('UAF snow depth [m]', fontsize=14)
ax[1,1].set_ylabel(' ')
ax[1,1].set_xlim([0, 1.5])
ax[1,1].set_ylim([0, 1.5])
ax[1,1].xaxis.set_major_formatter('{x:9<3.1f}')
ax[1,1].yaxis.set_major_formatter('{x:9<3.1f}')
ax[1,1].tick_params(axis='both', which='major', labelsize=12)
ax[1,1].annotate("r$^2$ = {:.2f}".format(linreg08.rvalue), (0.8, 0.15), fontsize=12)
ax[1,1].annotate("(e)", (0.05, 1.3), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[1,1].grid(True, alpha=0.3)
# ATL06-SR
sns.scatterplot(data=dfsr[(dfsr['beam']==50)&(dfsr['is2_height']>0)], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[1,2])
ax[1,2].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[1,2].set_xlabel(' ')
ax[1,2].set_ylabel(' ')
ax[1,2].set_xlim([0, 1.5])
ax[1,2].set_ylim([0, 1.5])
ax[1,2].xaxis.set_major_formatter('{x:9<3.1f}')
ax[1,2].yaxis.set_major_formatter('{x:9<3.1f}')
ax[1,2].tick_params(axis='both', which='major', labelsize=12)
ax[1,2].annotate("r$^2$ = {:.2f}".format(linregsr.rvalue), (0.8, 0.15), fontsize=12)
ax[1,2].annotate("(f)", (0.05, 1.3), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[1,2].grid(True, alpha=0.3)

## Histogram plots
#----------------#
# ATL06
sns.histplot(data=df06[(df06['beam']==5)], x='snow_depth_residual', ax=ax[2,0],
             bins=25,
             weights=w06)
ax[2,0].set_xlabel(' ')
ax[2,0].set_ylabel('Frequency', fontsize=14)
ax[2,0].xaxis.set_major_formatter('{x:9<3.1f}')
ax[2,0].set_xlim([-0.45, 0.45])
ax[2,0].set_ylim([0, 0.4])
ax[2,0].yaxis.grid(True)
ax[2,0].tick_params(axis='both', which='major', labelsize=12)
ax[2,0].annotate("Bias = {:.1f} cm".format(100*df06['snow_depth_residual'][df06['beam']==5].median()), (-0.3, 0.32), fontsize=12)
ax[2,0].annotate("NMAD = {:.0f} cm".format(100*nmad06), (-0.3, 0.27), fontsize=12)
ax[2,0].annotate("(g)", (-0.435, 0.355), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
# ATL08
sns.histplot(data=df08[df08['beam']==5], x='snow_depth_residual', ax=ax[2,1],
             bins=16,
             weights=w08)
ax[2,1].set_xlabel('IS2-UAF depth residual [m]', fontsize=14)
ax[2,1].set_ylabel(' ')
ax[2,1].xaxis.set_major_formatter('{x:9<3.1f}')
ax[2,1].set_xlim([-0.45, 0.45])
ax[2,1].set_ylim([0, 0.4])
ax[2,1].yaxis.grid(True)
ax[2,1].tick_params(axis='both', which='major', labelsize=12)
ax[2,1].annotate("Bias = {:.0f} cm".format(100*df08['snow_depth_residual'][df08['beam']==5].median()), (-0.3, 0.32), fontsize=12)
ax[2,1].annotate("NMAD = {:.0f} cm".format(100*nmad08), (-0.3, 0.27), fontsize=12)
ax[2,1].annotate("(h)", (-0.435, 0.355), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
# ATL06-SR
sns.histplot(data=dfsr[(dfsr['beam']==50)], x='snow_depth_residual', ax=ax[2,2],
             bins=20,
             weights=wsr)
ax[2,2].xaxis.set_major_formatter('{x:9<3.1f}')
ax[2,2].set_xlabel(' ')
ax[2,2].set_ylabel(' ')
ax[2,2].set_xlim([-0.45, 0.45])
ax[2,2].set_ylim([0, 0.4])
ax[2,2].yaxis.grid(True)
ax[2,2].tick_params(axis='both', which='major', labelsize=12)
ax[2,2].annotate("Bias = {:.1f} cm".format(100*dfsr['snow_depth_residual'][(dfsr['beam']==50)].median()), (-0.3, 0.32), fontsize=12)
ax[2,2].annotate("NMAD = {:.0f} cm".format(100*nmadsr), (-0.3, 0.27), fontsize=12)
ax[2,2].annotate("(i)", (-0.435, 0.355), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
plt.tight_layout()
#plt.savefig('/home/jovyan/icesat2-snowex/figures/cffl/is2_uaf_snowdepth_multipanel_rgt1356_cffl_2022321.png', dpi=500)

## Filtered FLCF Plot
Code to generate Figure 10 from Fair et al., (in prep). Creates a line plot, a scatter plot, and a histogram plot for ICESat-2/UAF snow depths, with ICESat-2 returns filtered by photon count (N > 110).

In [ ]:
# Calculate r2 value for filtered ATL06-SR returns
dfsr_r2 = dfsr[(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)].dropna()
linregsr = sp.stats.linregress(dfsr_r2['lidar_snow_depth'], dfsr_r2['is2_snow_depth'])

# Generate weights, NMAD for filtered ATL06-SR histograms
wsr = np.ones_like(dfsr['snow_depth_residual'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)])/float(len(dfsr['snow_depth_residual'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)]))
nmadsr = 1.4826*mad(dfsr['snow_depth_residual'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)].dropna())

fig, ax = plt.subplots(3, 1, figsize=(6,9))
## Line plot
#----------------#
ax[0].plot(dfsr['lat'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)], 
           dfsr['is2_snow_depth'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)], 
           label='ICESat-2', linewidth=3)
ax[0].plot(dfsr['lat'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)],
           dfsr['lidar_snow_depth'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)],
           label='UAF')
ax[0].set_xlabel('Latitude', fontsize=14)
ax[0].set_ylabel('Snow depth [m]', fontsize=14)
ax[0].set_xlim([64.861, 64.884])
ax[0].set_ylim([0, 1.75])
ax[0].yaxis.set_major_formatter('{x:9<3.1f}')
#ax[0].set_xticks([64.86, 64.865, 64.87, 64.875, 64.88, 64.885])
fig.tight_layout()
ax[0].grid(True)
ax[0].legend(loc='upper right')
ax[0].annotate("(a)", (64.8615, 1.56), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[0].tick_params(axis='both', which='major', labelsize=12)

## Scatter plot
#----------------#
sns.scatterplot(data=dfsr[(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)], x='lidar_snow_depth', y='is2_snow_depth', ax=ax[1])
ax[1].plot(np.linspace(0, 1.5, 25), np.linspace(0, 1.5, 25), color='black', linestyle='--', alpha=0.75)
ax[1].set_xlabel(' ')
ax[1].set_ylabel(' ')
ax[1].set_xlim([0, 1.5])
ax[1].set_ylim([0, 1.5])
ax[1].set_xlabel('UAF snow depth [m]', fontsize=14)
ax[1].set_ylabel('ICESat-2 snow depth [m]', fontsize=14)
ax[1].xaxis.set_major_formatter('{x:9<3.1f}')
ax[1].yaxis.set_major_formatter('{x:9<3.1f}')
ax[1].annotate("r$^2$ = {:.2f}".format(linregsr.rvalue), (1, 0.22), fontsize=12)
ax[1].annotate("(b)", (0.05, 1.335), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[1].grid(True, alpha=0.3)
ax[1].tick_params(axis='both', which='major', labelsize=12)
plt.tight_layout()

## Histogram plot
#----------------#
dfsr['snow_depth_residual'][dfsr['snow_depth_residual'].abs()>1] = np.nan
sns.histplot(data=dfsr[(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)], x='snow_depth_residual', ax=ax[2],
             bins=25,
             weights=wsr)
ax[2].xaxis.set_major_formatter('{x:9<3.1f}')
ax[2].set_xlabel(' ')
ax[2].set_ylabel(' ')
ax[2].set_xlabel('IS2-UAF depth residual [m]', fontsize=14)
ax[2].set_ylabel('Frequency', fontsize=14)
ax[2].set_xlim([-0.5, 0.5])
ax[2].yaxis.grid(True)
ax[2].annotate("Bias = {:.1f} cm".format(100*dfsr['snow_depth_residual'][(dfsr['beam']==50)&(dfsr['n_fit_photons']>=110)].median()), (0.2, 0.155), fontsize=12)
ax[2].annotate("NMAD = {:.1f} cm".format(100*nmadsr), (0.2, 0.135), fontsize=12)
ax[2].annotate("(c)", (-0.47, 0.17), fontsize=16, bbox=dict(facecolor='white', alpha=0.5))
ax[2].tick_params(axis='both', which='major', labelsize=12)
plt.tight_layout()

## Signal Photon KDE Plot
Generates Figure 9 from Fair et al., (in prep). Plots the number of signal photons from ATL06-SR segments as KDE plots by site (a), all UKT tracks (b), and all FLCF tracks (c).

In [ ]:
# Create GeoDataFrame of FLCF data
df = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgtall_cffl_2022-2023_442010.csv')
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'], crs="EPSG:4326")

# Do the same for UKT
df_utk = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgtall_utk_2022-2023_442010.csv')
df_utk['geometry'] = df_utk['geometry'].apply(wkt.loads)
gdf_utk = gpd.GeoDataFrame(df_utk, geometry=df_utk['geometry'], crs="EPSG:4326")

In [ ]:
# Add "month" column to each DataFrame
gdf['month'] = pd.to_datetime(gdf['time']).dt.month
gdf_utk['month'] = pd.to_datetime(gdf_utk['time']).dt.month

In [ ]:
# Determine yaw orientation of ICESat-2
gdf_preflip = gdf[gdf['month']<6]
gdf_postflip = gdf[gdf['month']>=6]
gdf_preflip_utk = gdf_utk[gdf_utk['month']<6]
gdf_postflip_utk = gdf_utk[gdf_utk['month']>=6]

# Create DataFrames of strong beams only, based on yaw orientation (FLCF)
try:
    gdf_preflip_strong = gdf_preflip[(gdf_preflip['beam']==10)|(gdf_preflip['beam']==30)|(gdf_preflip['beam']==50)]
    gdf_postflip_strong = gdf_postflip[(gdf_postflip['beam']==20)|(gdf_postflip['beam']==40)|(gdf_postflip['beam']==60)]
except:
    gdf_preflip_strong = gdf_preflip[(gdf_preflip['gt']==10)|(gdf_preflip['gt']==30)|(gdf_preflip['gt']==50)]
    gdf_postflip_strong = gdf_postflip[(gdf_postflip['gt']==20)|(gdf_postflip['gt']==40)|(gdf_postflip['gt']==60)]

# Create DataFrames of strong beams only, based on yaw orientation (UKT)
try:
    gdf_preflip_strong_utk = gdf_preflip_utk[(gdf_preflip_utk['beam']==10)|(gdf_preflip_utk['beam']==30)|(gdf_preflip_utk['beam']==50)]
    gdf_postflip_strong_utk = gdf_postflip_utk[(gdf_postflip_utk['beam']==20)|(gdf_postflip_utk['beam']==40)|(gdf_postflip_utk['beam']==60)]
except:
    gdf_preflip_strong_utk = gdf_preflip_utk[(gdf_preflip_utk['gt']==10)|(gdf_preflip_utk['gt']==30)|(gdf_preflip_utk['gt']==50)]
    gdf_postflip_strong_utk = gdf_postflip_utk[(gdf_postflip_utk['gt']==20)|(gdf_postflip_utk['gt']==40)|(gdf_postflip_utk['gt']==60)]

# Concatenate pre-flip, post-flip dataframes for each site
gdf_strong = pd.concat([gdf_preflip_strong, gdf_postflip_strong])
gdf_strong_utk = pd.concat([gdf_preflip_strong_utk, gdf_postflip_strong_utk])

In [ ]:
# Generate weights for KDE plots
wm = np.ones_like(np.array(gdf_mar2022['n_fit_photons']))/float(len(np.array(gdf_mar2022['n_fit_photons'])))
wo = np.ones_like(np.array(gdf_oct['n_fit_photons']))/float(len(np.array(gdf_oct['n_fit_photons'])))
ws = np.ones_like(np.array(gdf_so['n_fit_photons']))/float(len(np.array(gdf_so['n_fit_photons'])))

fig, ax = plt.subplots(1,3, sharex=True, figsize=(12, 6))
## Site comparison KDE
#-----------------------#
sns.kdeplot(data=gdf_strong, x='n_fit_photons', ax=ax[0], color='#2ca02c',
            alpha=0.5, label='FLCF', fill=True)
sns.kdeplot(data=gdf_strong_utk, x='n_fit_photons', ax=ax[0], color='#ff7f0e',
            alpha=0.5, label='UKT', fill=True)
ax[0].set_xlim([0, 250])
ax[0].set_xlabel(' ', fontsize=16)
ax[0].set_ylabel('Density', fontsize=16)
ax[0].set_title('All tracks', fontsize=14)
ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].yaxis.grid(True)
ax[0].legend()
ax[0].annotate("(a)", (10, 0.0093), fontsize=16, bbox=dict(facecolor='white', alpha=0.8))

## UKT KDE
#----------------#
sns.kdeplot(data=gdf_mar2022_utk, x='n_fit_photons', ax=ax[1], color='#2ca02c',
             alpha=0.5, zorder=1, label='Jan-Apr 2023', fill=True)
sns.kdeplot(data=gdf_oct_utk, x='n_fit_photons', ax=ax[1], color='#ff7f0e',
             alpha=0.5, zorder=2, label='Sep-Dec 2022', fill=True)
sns.kdeplot(data=gdf_so_utk, x='n_fit_photons', ax=ax[1], color='#1f77b4',
             alpha=0.5, zorder=3, label='May-Aug 2022', fill=True)
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel('Number of signal photons, per segment', fontsize=16)
ax[1].set_xlim([0, 250])
ax[1].set_xlabel(' ', fontsize=16)
ax[1].set_ylabel(' ', fontsize=16)
ax[1].set_title('Upper Kuparuk/Toolik', fontsize=14)
ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].yaxis.grid(True)
ax[1].legend()
ax[1].annotate("(b)", (10, 0.022), fontsize=16, bbox=dict(facecolor='white', alpha=0.8))

## FLCF KDE
#----------------#
sns.kdeplot(data=gdf_mar2022, x='n_fit_photons', ax=ax[2], color='#2ca02c',
             alpha=0.5, zorder=1, label='Jan-Apr 2023', fill=True)
sns.kdeplot(data=gdf_oct, x='n_fit_photons', ax=ax[2], color='#ff7f0e',
             alpha=0.5, zorder=2, label='Sep-Dec 2022', fill=True)
sns.kdeplot(data=gdf_so, x='n_fit_photons', ax=ax[2], color='#1f77b4',
             alpha=0.5, zorder=3, label='May-Aug 2022', fill=True)
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel('Number of signal photons, per segment', fontsize=16)
ax[2].set_xlim([0, 250])
ax[2].set_xlabel(' ', fontsize=16)
ax[2].set_ylabel(' ', fontsize=16)
ax[2].set_title("Farmer's Loop/Creamer's Field", fontsize=14)
ax[2].tick_params(axis='both', which='major', labelsize=12)
ax[2].yaxis.grid(True)
ax[2].legend()
ax[2].annotate("(c)", (10, 0.034), fontsize=16, bbox=dict(facecolor='white', alpha=0.8))
handles, labels = ax[1].get_legend_handles_labels()
plt.tight_layout()

## ATL06-SR Resolution Box Plots
Generates Figure 8 from Fair et al., (in prep). Creates two box plots that relate IS2-UAF depth residuals to ATL06-SR resolution over FLCF (a) and UKT (b).

In [ ]:
# Initialize DataFrames at 10 m resolution
dfsr_44105 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_44105.csv')
dfsr_utk_44105 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgt152_utk_2023331_44105.csv')

In [ ]:
# Create Xarrays to coarsen the data to 20m/40m/100m (FLCF)
interp_coarse = dfsr_44105[dfsr_44105['beam']==50][['lat', 'lon', 'lidar_snow_depth', 'is2_snow_depth', 'snow_depth_residual']]
interp_coarse_10m = interp_coarse.set_index(['lat', 'lon']).to_xarray()
interp_coarse_20m = interp_coarse.set_index(['lat', 'lon']).to_xarray().coarsen(lat=2, lon=2, boundary='trim').median()
interp_coarse_40m = interp_coarse.set_index(['lat', 'lon']).to_xarray().coarsen(lat=4, lon=4, boundary='trim').median()
interp_coarse_100m = interp_coarse.set_index(['lat', 'lon']).to_xarray().coarsen(lat=10, lon=10, boundary='trim').median()

# Do the same for UKT
interp_coarse_utk = dfsr_utk_44105[dfsr_utk_44105['beam']==50][['lat', 'lon', 'lidar_snow_depth', 'is2_snow_depth', 'snow_depth_residual']]
interp_coarse_utk_10m = interp_coarse_utk.set_index(['lat', 'lon']).to_xarray()
interp_coarse_utk_20m = interp_coarse_utk.set_index(['lat', 'lon']).to_xarray().coarsen(lat=2, lon=2, boundary='trim').median()
interp_coarse_utk_40m = interp_coarse_utk.set_index(['lat', 'lon']).to_xarray().coarsen(lat=4, lon=4, boundary='trim').median()
interp_coarse_utk_100m = interp_coarse_utk.set_index(['lat', 'lon']).to_xarray().coarsen(lat=10, lon=10, boundary='trim').median()

In [ ]:
# Convert resolution-separated data into DataFrames, and add resolution as a label (FLCF)
interp_coarse_10m_pd = interp_coarse_10m.to_dataframe().dropna()
interp_coarse_10m_pd['filter'] = '10 m'
interp_coarse_20m_pd = interp_coarse_20m.to_dataframe().dropna()
interp_coarse_20m_pd['filter'] = '20 m'
interp_coarse_40m_pd = interp_coarse_40m.to_dataframe().dropna()
interp_coarse_40m_pd['filter'] = '40 m'
interp_coarse_100m_pd = interp_coarse_100m.to_dataframe().dropna()
interp_coarse_100m_pd['filter'] = '100 m'

# Do the same for UKT
interp_coarse_utk_10m_pd = interp_coarse_utk_10m.to_dataframe().dropna()
interp_coarse_utk_10m_pd['filter'] = '10 m'
interp_coarse_utk_20m_pd = interp_coarse_utk_20m.to_dataframe().dropna()
interp_coarse_utk_20m_pd['filter'] = '20 m'
interp_coarse_utk_40m_pd = interp_coarse_utk_40m.to_dataframe().dropna()
interp_coarse_utk_40m_pd['filter'] = '40 m'
interp_coarse_utk_100m_pd = interp_coarse_utk_100m.to_dataframe().dropna()
interp_coarse_utk_100m_pd['filter'] = '100 m'

In [ ]:
# Concatenate DataFrames
interp_coarse_allres = pd.concat([interp_coarse_10m_pd, interp_coarse_20m_pd, interp_coarse_40m_pd, interp_coarse_100m_pd])
interp_coarse_utk_allres = pd.concat([interp_coarse_utk_10m_pd, interp_coarse_utk_20m_pd, interp_coarse_utk_40m_pd, interp_coarse_utk_100m_pd])

# Melt DataFrames to sort them by resolution ("filter")
interp_coarse_allres_melt = interp_coarse_allres.reset_index().melt(id_vars=['filter'], value_vars=['snow_depth_residual'])
interp_coarse_utk_allres_melt = interp_coarse_utk_allres.reset_index().melt(id_vars=['filter'], value_vars=['snow_depth_residual'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14,8), sharex=True)

## FLCF Boxplot
#----------------#
sns.boxplot(ax=ax[0], data=interp_coarse_allres_melt, x='filter', y='value')
ax[0].set_ylim([-1, 1])
ax[0].set_xlabel(' ')
ax[0].set_ylabel('IS2-UAF depth residual [m]', fontsize=16)
ax[0].set_title("Creamer's Field", fontsize=14)
ax[0].tick_params(axis='both', which='major', labelsize=14)
ax[0].grid(True)

## UKT Boxplot
#----------------#
sns.boxplot(ax=ax[1], data=interp_coarse_utk_allres_melt, x='filter', y='value')
ax[1].set_ylim([-1, 1])
ax[1].set_xlabel(' ')
ax[1].set_ylabel(' ')
ax[1].set_title("Upper Kuparuk/Toolik", fontsize=14)
ax[1].tick_params(axis='both', which='major', labelsize=14)
ax[1].grid(True)

fig.supxlabel('SlideRule resolution [m]', fontsize=16)
t1 = ax[0].text(-0.4, -0.95, '(a)', fontsize=24)
t1.set_bbox(dict(facecolor='white', alpha=0.75, edgecolor='white'))
t2 = ax[1].text(-0.4, -0.95, '(b)', fontsize=24)
t2.set_bbox(dict(facecolor='white', alpha=0.75, edgecolor='white'))
plt.tight_layout()

## ATL06-SR Photon Filter Boxplots
Creates Figure 7 from Fair et al., (in prep). Generates boxplots of snow depth residuals as a function of photon filter. The filtering configurations are given below:

* "High, Ground": High-confidence photons only, ATL08 ground photons only
* "High, All": High-confidence photons only, no ATL08 filter
* "All, Ground": No signal confidence filter (except noise), ATL08 ground photons only
* "All, All": No signal confidence filter (except noise), no ATL08 filter

In [ ]:
# Initialize DataFrames (FLCF)
dfsr_442010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_442010.csv')
dfsr_412010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_4-12010.csv')
dfsr_142010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_142010.csv')
dfsr_112010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_1-12010.csv')

# Do the same for UKT
dfsr_utk_442010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgt152_utk_2023331_442010.csv')
dfsr_utk_412010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgt152_utk_2023331_4-12010.csv')
dfsr_utk_142010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgt152_utk_2023331_142010.csv')
dfsr_utk_112010 = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgt152_utk_2023331_1-12010.csv')

In [ ]:
# Add photon filter as label for DataFrame (FLCF)
dfsr_442010['filter'] = 'High, Ground'
dfsr_142010['filter'] = 'All, Ground'
dfsr_412010['filter'] = 'High, All'
dfsr_112010['filter'] = 'All, All'

# Do the same for UKT
dfsr_utk_442010['filter'] = 'High, Ground'
dfsr_utk_142010['filter'] = 'All, Ground'
dfsr_utk_412010['filter'] = 'High, All'
dfsr_utk_112010['filter'] = 'All, All'

In [ ]:
# Concatenate DataFrames
df_allfilters = pd.concat([dfsr_442010, dfsr_412010, dfsr_142010, dfsr_112010])
df_utk_allfilters = pd.concat([dfsr_utk_442010, dfsr_utk_412010, dfsr_utk_142010, dfsr_utk_112010])

# Melt DataFrames to sort them by photon filter ("filter")
df_allfilters_melt = df_allfilters.reset_index().melt(id_vars=['filter'], value_vars=['snow_depth_residual'])
df_utk_allfilters_melt = df_utk_allfilters.reset_index().melt(id_vars=['filter'], value_vars=['snow_depth_residual'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14,8), sharex=True)

## FLCF Boxplot
#----------------#
sns.boxplot(ax=ax[0], data=df_allfilters_melt, x='filter', y='value')
ax[0].set_ylim([-1, 1])
ax[0].set_xlabel(' ', fontsize=16)
ax[0].set_ylabel('IS2-UAF depth residual [m]', fontsize=16)
ax[0].set_title("Creamer's Field", fontsize=14)
ax[0].tick_params(axis='both', which='major', labelsize=14)
ax[0].grid(True)

## UKT Boxplot
#----------------#
sns.boxplot(ax=ax[1], data=df_utk_allfilters_melt, x='filter', y='value')
ax[1].set_ylim([-1, 1])
ax[1].set_xlabel(' ')
ax[1].set_ylabel(' ')
ax[1].set_title("Upper Kuparuk/Toolik", fontsize=14)
ax[1].tick_params(axis='both', which='major', labelsize=14)
ax[1].grid(True)

fig.supxlabel('Signal confidence, ATL08 class [m]', fontsize=16)
t1 = ax[0].text(-0.4, -0.95, '(a)', fontsize=24)
t1.set_bbox(dict(facecolor='white', alpha=0.75, edgecolor='white'))
t2 = ax[1].text(-0.4, -0.95, '(b)', fontsize=24)
t2.set_bbox(dict(facecolor='white', alpha=0.75, edgecolor='white'))
plt.tight_layout()

## SNOTEL Time Series
Creates Figure 4 from Fair et al., (in prep). Plots SNOTEL snow depths over time, with the acquisition dates of UAF and ICESat-2 also given.

In [ ]:
# Read UKT SNOTEL data
ukt = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/station-data/imnaviat-creek-annual-snow-depth-plot.csv')
ukt_2023 = ukt['2022-2023']
ukt_2023_df = pd.DataFrame({'daynum':ukt_2023.index+1, 'snow_depth_meters': ukt_2023.values*0.0254})

# Read FLCF SNOTEL data
cffl = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/station-data/snotel_cffl_2022-2023.csv')
cffl['year-month'] = pd.to_datetime(cffl['Date']).dt.strftime('%Y-%m')
cffl_mar22 = cffl[cffl['year-month']=='2022-03']
cffl_mar22['snow_depth_meters'] = cffl_mar22['SNWD.I-1 (in) ']*0.0254

# Add year-month-day to DataFrames
ukt_2023_df['year-month-day'] = pd.to_datetime(ukt_2023_df['daynum'], unit='D', origin=pd.Timestamp('2022-09-01'))
ukt_2023_df['year-month'] = pd.to_datetime(ukt_2023_df['year-month-day']).dt.strftime('%Y-%m')
ukt_mar2023_df = ukt_2023_df[ukt_2023_df['year-month']=='2023-03']
cffl_mar22['year-month-day'] = pd.to_datetime(cffl_mar22['Date']).dt.strftime('%Y-%m-%d')

In [ ]:
# Load the relevant UAF data
uaf_cffl = rxr.open_rasterio('/home/jovyan/icesat2-snowex/lidar-dems/farmersloop_2022mar11_snowdepth_3m.tif')
uaf_ukt = rxr.open_rasterio('/home/jovyan/icesat2-snowex/lidar-dems/UKT_2023mar13_snowdepth_3m.tif')

# Load the relevant ICESat-2 data
is2_cffl = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/cffl/atl06sr_snowdepth_rgt1356_cffl_2022321_444020.csv')
is2_ukt = pd.read_csv('/home/jovyan/icesat2-snowex/snow-depth-data/utk/atl06sr_snowdepth_rgt152_utk_2023331_444020.csv')

In [ ]:
# Compute median bias, NMAD for UAF snow depth
uaf_cffl_median = uaf_cffl.where(uaf_cffl>0).median()
uaf_ukt_median = uaf_ukt.where(uaf_ukt>0).median()
uaf_cffl_nmad = 1.4826*mad(uaf_cffl.where(uaf_cffl>0).values.ravel(), nan_policy='omit')
uaf_ukt_nmad = 1.4826*mad(uaf_ukt.where(uaf_ukt>0).values.ravel(), nan_policy='omit')

# Do the same for ICESat-2
is2_cffl_median = is2_cffl[is2_cffl['beam']==50]['is2_snow_depth'].median()
is2_ukt_median = is2_ukt[is2_ukt['beam']==10]['is2_snow_depth'].median()
is2_cffl_nmad = 1.4826*mad(is2_cffl[is2_cffl['beam']==50]['is2_snow_depth'])
is2_ukt_nmad = 1.4826*mad(is2_ukt[is2_ukt['beam']==10]['snow_depth_residual'])

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(9, 6))

## UKT Time Series
#----------------#
axs[0].plot(ukt_mar2023_df['year-month-day'], ukt_mar2023_df['snow_depth_meters'], color='black')
axs[0].errorbar(pd.to_datetime('2023-03-13'), uaf_ukt_median, 
                yerr=uaf_ukt_nmad, fmt='o', capsize=4, color='orange')
axs[0].errorbar(pd.to_datetime('2023-03-31'), is2_ukt_median, 
                yerr=is2_ukt_nmad, fmt='o', capsize=4, color='blue')
axs[0].set_xticks(['2023-03-01', '2023-03-15', '2023-03-31'])
axs[0].tick_params(axis='both', which='major', labelsize=12)
#axs[0].axvline(x=pd.to_datetime('2023-03-13'), ls='--', lw=3, color='orange')
#axs[0].axvline(x=pd.to_datetime('2023-03-31'), ls='--', lw=3)
axs[0].text(pd.to_datetime('2023-03-21'), 0.31, 'ICESat-2\noverpass', fontsize=16)
axs[0].text(pd.to_datetime('2023-03-14'), 0.38, 'UAF\nflight', fontsize=16)
axs[0].annotate("(a)", (pd.to_datetime('2023-03-01'), 0.26), fontsize=16, bbox=dict(facecolor='white', alpha=0.7))
axs[0].grid(alpha=0.5)
axs[0].set_ylim([0.25, 0.625])
axs[0].set_ylabel('SNOTEL snow depth [m]', fontsize=18)
axs[0].set_title('Upper Kuparuk/Toolik', fontsize=16)

## FLCF Time Series
#----------------#
axs[1].plot(pd.to_datetime(cffl_mar22['year-month-day']), cffl_mar22['snow_depth_meters'], color='black')
axs[1].errorbar(pd.to_datetime('2022-03-11'), uaf_cffl_median, 
                yerr=uaf_cffl_nmad, fmt='o', capsize=4, color='orange')
axs[1].errorbar(pd.to_datetime('2022-03-21'), is2_cffl_median, 
                yerr=is2_cffl_nmad, fmt='o', capsize=4, color='blue')
axs[1].set_xticks(['2022-03-01', '2022-03-15', '2022-03-31'])
axs[1].tick_params(axis='both', which='major', labelsize=12)
#axs[1].axvline(x=pd.to_datetime('2022-03-11'), ls='--', lw=3, color='orange')
#axs[1].axvline(x=pd.to_datetime('2022-03-21'), ls='--', lw=3)
axs[1].text(pd.to_datetime('2022-03-22'), 0.88, 'ICESat-2\noverpass', fontsize=16)
axs[1].text(pd.to_datetime('2022-03-12'), 0.82, 'UAF\nflight', fontsize=16)
axs[1].annotate("(b)", (pd.to_datetime('2022-03-01'), 0.657), fontsize=16, bbox=dict(facecolor='white', alpha=0.7))
axs[1].grid(alpha=0.5)
axs[1].set_ylim([0.65, 1])
axs[1].set_title("Farmer's Loop/Creamer's Field", fontsize=16)
plt.tight_layout()